In [ ]:
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
tf.compat.v1.enable_v2_behavior()

RANDOM_SEED = 42

In [ ]:
dataset = './hand_sign.csv'
model_save_path = './hand_sign.hdf5'
tflite_save_path = './hand_sign.tflite'

## Ler input

In [ ]:
dict = {
        97  : "a",
        98  : "b",
        99  : "c",
        100 : "d",
        101 : "e",
        102 : "f",
        103 : "g",
        104 : "h",
        105 : "i",
        106 : "j",
        107 : "k",
        108 : "l",
        109 : "m",
        110 : "n",
        111 : "o",
        112 : "p",
        113 : "q",
        114 : "r",
        115 : "s",
        116 : "t",
        117 : "u",
        118 : "v",
        119 : "w",
        120 : "x",
        121 : "y",
        122 : "z"
}
NUM_CLASSES = len(dict.keys())
print(NUM_CLASSES)
datasetLetters = []

for i in dict.keys():
    datasetLetters.append(f'./hand_sign_{dict[i]}.csv')

# Abre um novo arquivo para escrita
with open(dataset, 'w', newline='') as arquivo_final:
    # Loop pelos arquivos CSV
    for arquivo in datasetLetters:
        # Abre o arquivo CSV
        try:
          with open(arquivo, 'r') as arquivo_atual:
              # Lê cada linha e escreve no novo arquivo
              for linha in arquivo_atual:
                  arquivo_final.write(linha)
        except:
          continue

26


# Train test split

In [ ]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.8, random_state=RANDOM_SEED)


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_8 (Dropout)         (None, 42)                0         
                                                                 
 dense_12 (Dense)            (None, 20)                860       
                                                                 
 dropout_9 (Dropout)         (None, 20)                0         
                                                                 
 dense_13 (Dense)            (None, 10)                210       
                                                                 
 dense_14 (Dense)            (None, 26)                286       
                                                                 
Total params: 1356 (5.30 KB)
Trainable params: 1356 (5.30 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Treino

In [ ]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    X_train,
    y_train,
    epochs=150,
    batch_size=32,
    validation_data=(X_test, y_test)
)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)

    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)

    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()

    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

## Salvando modelo

In [ ]:
model.save(model_save_path, include_optimizer=False)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Salvando modelo TFLite

Essa parte é necessária para melhor integrar com o mediapipe enquanto a aplicação tiver rodando. Isso foi retirado da biblioteca do tensorflow e também [desse repositório](https://github.com/kinivi/hand-gesture-recognition-mediapipe).

In [ ]:
run_model = tf.function(lambda x: model(x))
BATCH_SIZE = 1
INPUT_SIZE = 42
concrete_func = run_model.get_concrete_function(tf.TensorSpec([BATCH_SIZE, INPUT_SIZE], model.inputs[0].dtype))
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]

In [ ]:
tflite_quantized_model = converter.convert()
f = open(tflite_save_path, 'wb')
f.write(tflite_quantized_model)
f.close()